# Langchain Learning


In [25]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

## LLMs


In [3]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

/Users/meet/anaconda3/envs/omdena/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/meet/anaconda3/envs/omdena/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




"Royal Spice House"


In [3]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\nSaffron & Spice.'

## Prompt Templates


In [14]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains


In [15]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

/Users/meet/anaconda3/envs/omdena/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"El Sabor Exquisito" (The Exquisite Flavor)'

In [27]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"El Sabor Exquisito" (The Exquisite Flavor)'

In [17]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain


In [18]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)

 

1. Tandoori Chicken: Tender chicken marinated in aromatic spices and roasted in a traditional clay oven.

2. Vegetable Samosas: Crispy pastries filled with a savory mix of potatoes, peas, and spices.

3. Lamb Vindaloo: A spicy and tangy curry dish made with tender chunks of lamb, potatoes, and a blend of Indian spices.

4. Palak Paneer: Creamy spinach curry with cubes of paneer (Indian cottage cheese) and aromatic spices.

5. Butter Chicken: A popular Indian dish of tender chicken cooked in a rich and creamy tomato-based sauce.

6. Chana Masala: A flavorful and hearty dish made with chickpeas, tomatoes, onions, and a blend of spices.

7. Garlic Naan: Soft and fluffy flatbread topped with garlic and herbs, perfect for dipping in curries.

8. Chicken Tikka Masala: Tender pieces of chicken cooked in a rich and creamy tomato-based sauce with a blend of spices.

9. Aloo Gobi: A vegetarian dish made with potatoes, cauliflower, and a mix of spices, perfect for those looking for a milder op

#### Sequential Chain


In [19]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [20]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [21]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': "\n\nMaharaja's Spice Palace",
 'menu_items': '\n\n- Chicken Tikka Masala\n- Vegetable Biryani\n- Lamb Rogan Josh\n- Paneer Makhani\n- Aloo Gobi\n- Palak Paneer\n- Chana Masala\n- Vegetable Korma\n- Samosas\n- Naan Bread\n- Papadums\n- Raita\n- Mango Lassi'}

## Agents


In [22]:
# make sure yoy have installed this package: pip install google-search-results
from secret_key import serpapi_key
os.environ['SERPAPI_API_KEY'] = serpapi_key



In [30]:
from langchain_community.utilities import SerpAPIWrapper
search = SerpAPIWrapper()
search.run("Obama's first name?")

'Barack Hussein Obama II'

In [32]:
from langchain.agents import load_tools,initialize_agent,AgentType

tools = load_tools(["serpapi"])
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5trillion?")




> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: "GDP of US in 2022"
Observation: $25.46 trillion
Thought: I should add 5 trillion to the GDP.
Action: Addition
Action Input: $25.46 trillion + $5 trillion
Observation: Addition is not a valid tool, try one of [Search].
Thought: I should use a calculator to add the numbers.
Action: Search
Action Input: "calculator"
Observation: 0
Thought: I should use a calculator to add the numbers.
Action: Search
Action Input: "calculator"
Observation: 0
Thought:

KeyboardInterrupt: 

#### serpapi and llm-math tool


In [28]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I should use a calculator to add 5 to the GDP of US in 2022
Action: Calculator
Action Input: GDP of US in 2022 + 5

ValueError: LLMMathChain._evaluate("
GDP_US_2022 + 5
") raised error: 'GDP_US_2022'. Please try again with a valid numerical expression

#### Wikipedia and llm-math tool


In [29]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Calculator to calculate Elon Musk's age in 2023.
Action: Calculator
Action Input: 2023 - 1971
Observation: Answer: 52
Thought: I now know the final answer.
Final Answer: Elon Musk was born on June 28, 1971, so his age in 2023 will be 52.

> Finished chain.


'Elon Musk was born on June 28, 1971, so his age in 2023 will be 52.'

## Memory


In [17]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [18]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [19]:
chain.memory

In [20]:
type(chain.memory)

NoneType

#### ConversationBufferMemory


In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [22]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [23]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain


In [24]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [25]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [26]:
convo.run("How much is 5+5?")

' 10.'

In [27]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.'

In [29]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory


In [30]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and the winner was the West Indies.'

In [31]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [32]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."